In [5]:
import geopandas as gpd
import geoai

In [9]:
satellite_imagery = './pabbi_satellite_image_COG.tif'
vector_file = './pabb_crop_V3.geojson'

In [15]:

gdf = gpd.read_file(vector_file)

print(f"Total Number of Parcels {len(gdf)}")

print(f"Number of Parcels with values (other than null): {gdf['Landuse_Ma'].notna().sum()}")

print(f"Details of Crop are : {gdf['Landuse_Ma'].value_counts()}")

Total Number of Parcels 7012
Number of Parcels with values (other than null): 6515
Details of Crop are : Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: count, dtype: int64


In [16]:
label_map = {
    'Agriculture': 1,
    'Stream': 2,
    'Other': 3,
    'Road/Streets': 4,
    'Graveyard': 5,
    'Built up': 6
}

gdf['class_id'] = gdf['Landuse_Ma'].map(label_map)


#Vector to Raster does not work as expected so we changed the approach and generated the label raster manually. 
geoai.vector_to_raster(
    vector_path=vector_file,
    reference_raster=satellite_imagery,
    attribute_field='class_id',
    output_path='Labelraster_For_Unet_v3.tif',
    dtype='uint8',
    plot_result=True,
    nodata=0
    )

In [6]:
vector_path = 'temp_cleaned_vector.geojson'
labeled_raster_path = 'labelraster_Manual.tif'
out_folder = 'landcover_pabbi_satellite'

In [ ]:
# The next step was to form tiles of the satellite and label image.
out_folder = "landcover_satellite_tiles"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_landcover_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

In [ ]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=4,
    num_classes=13,
    batch_size=4,
    num_epochs=2,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
    plot_curves=True,
)

Using device: cpu
Found 4477 image files and 4477 label files
Training on 3581 images, validating on 896 images
Starting training with unet + resnet34
Model parameters: 24,441,245
